In [2]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
from itertools import product
import io

df1 = pd.read_csv("./data/student_dataset.csv")
df2 = pd.read_csv("./data/scholarship_dataset.csv")

df3=df1.merge(df2, how='cross')


In [4]:
def generate_output(row):
    if (((row['Education_Level'] =='BS') and (row['Education_Preference']=='Masters')) or ((row['Education_Level'] =='Masters') and (row['Education_Preference']=='PhD')) or ((row['Education_Level'] =='PhD') and (row['Education_Preference']=='PhD'))) and (row['Domain_Area'] == row['Eligible_Domain']) and (row['Country_Preference'] == row['Country_of_Scholarship']):
        return 1
    else:
        return 0

In [5]:
df3['Target'] = df3.apply(generate_output, axis=1)

df3.to_csv("combined_dataset.csv", index=False)

In [6]:
print(df3['Target'].value_counts())

Target
0    248045
1      1955
Name: count, dtype: int64


In [7]:
df3 = pd.read_csv("combined_dataset.csv")

# Split the dataset into features (X) and target variable (y)
X = df3.drop('Target', axis=1)
y = df3['Target']

# One-hot encode categorical columns
categorical_columns = ["Name", "Education_Level", "Country_Preference", "Domain_Area", "Scholarship_Name",
                       "Scholarship_Type", "Education_Preference", "Country_of_Scholarship",
                       "Eligible_Domain"]

X = pd.get_dummies(X, columns=categorical_columns)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Build a simple neural network model
model = Sequential([
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with a specified number of epochs
epochs = 5  # Adjust the number of epochs as needed
model.fit(X_train_scaled, y_train, epochs=epochs, validation_data=(X_test_scaled, y_test))

Epoch 1/5
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9386 - loss: 0.1464 - val_accuracy: 0.9926 - val_loss: 0.0423
Epoch 2/5
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9920 - loss: 0.0393 - val_accuracy: 0.9926 - val_loss: 0.0362
Epoch 3/5
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9926 - loss: 0.0285 - val_accuracy: 0.9933 - val_loss: 0.0286
Epoch 4/5
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9935 - loss: 0.0197 - val_accuracy: 0.9941 - val_loss: 0.0218
Epoch 5/5
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9948 - loss: 0.0109 - val_accuracy: 0.9961 - val_loss: 0.0167


In [2]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predict probabilities for binary classification
y_pred_prob = model.predict(X_test_scaled)

# Convert probabilities to binary labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Ensure y_test and y_pred have the same data type and shape
y_test = np.array(y_test)
y_pred = np.array(y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Generate classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Generate confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


NameError: name 'model' is not defined

In [10]:
import keras
keras.saving.save_model(model,"recommender-system.keras")